## 前処理

In [1]:
# Porchのインストール
import torch

# cudaが使えるなら使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
# 学習データ（例）
sample_texts = [
    "Hello, how are you doing today?",
    "Hello, how are you?",
    "Hello, how are you?",
    "How are you doing?",
    "This is a small example dataset for BERT.",
    "We will train a mini BERT model to learn masked language modeling.",
    "Large models usually require huge datasets and computational resources.",
    "But here, we will just use a small corpus.",
    "Hello, how are you doing again today?",
    "Hello, how are you doing once more?",
	  "Hello, how are you doing?",
]

In [3]:
def simple_tokenize(sentence):
	  # 文の中の句読点を削除
    for p in [".", ",", "?", "!", ":", ";"]:
        sentence = sentence.replace(p, "")
  	# 文をスペースで分割してトークン化
    tokens = sentence.strip().split()
    return tokens

# sample_textsの各文にsimple_tokenize関数を適用
# 結果は、各文のトークンを保持するリストのリスト
tokenized_texts = [simple_tokenize(t) for t in sample_texts] # 内包表記
tokenized_texts

[['Hello', 'how', 'are', 'you', 'doing', 'today'],
 ['Hello', 'how', 'are', 'you'],
 ['Hello', 'how', 'are', 'you'],
 ['How', 'are', 'you', 'doing'],
 ['This', 'is', 'a', 'small', 'example', 'dataset', 'for', 'BERT'],
 ['We',
  'will',
  'train',
  'a',
  'mini',
  'BERT',
  'model',
  'to',
  'learn',
  'masked',
  'language',
  'modeling'],
 ['Large',
  'models',
  'usually',
  'require',
  'huge',
  'datasets',
  'and',
  'computational',
  'resources'],
 ['But', 'here', 'we', 'will', 'just', 'use', 'a', 'small', 'corpus'],
 ['Hello', 'how', 'are', 'you', 'doing', 'again', 'today'],
 ['Hello', 'how', 'are', 'you', 'doing', 'once', 'more'],
 ['Hello', 'how', 'are', 'you', 'doing']]

## 辞書の作成

In [5]:
from collections import Counter

# 全てのトークンを保存
all_tokens = []
for tokens in tokenized_texts:
    all_tokens.extend(tokens)

# それぞれのトークンの出現回数をカウント
# Counterでユニークな数をカウント
counts = Counter(all_tokens)

# 特殊トークンを定義
# [PAD]はシーケンスの長さを揃えるために使用
# [UNK]は語彙にない未知のトークン用
# [CLS]は文の開始を示すために使用
# [SEP]は文の終了を示すために使用
# [MASK]はMLM用
# その後、通常のトークンをアルファベット順にソートして追加
vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + sorted(counts)

In [6]:
# 2つの辞書を作成
# word2idx: トークン文字列 -> 整数インデックス
# idx2word: 整数インデックス -> トークン文字列
word2idx = {w: i for i, w in enumerate(vocab)} # 内包表記で辞書を作成
idx2word = {i: w for w, i in word2idx.items()} # 数値化されたトークンを元に戻すことが可能

# vocab_sizeは、特殊トークンを含むユニークなトークンの数
vocab_size = len(vocab)
print(f"Vocab size: {vocab_size}")

Vocab size: 48


In [7]:
# トークンのリストを整数IDのリストに変換
# - [CLS]を先頭に、[SEP]を末尾に追加
# - 各トークンを整数IDに変換
# - max_lenに合わせて[PAD]でパディングまたはトランケート
def encode(tokens, word2idx, max_len=12): # 有効なトークンを作成する関数

  	# [CLS]は文の開始を示し、[SEP]は文の終了を示す
    tokens = ["[CLS]"] + tokens + ["[SEP]"]

  	# word2idx辞書を使ってトークンを整数IDに変換
    token_ids = []
    for t in tokens:
	    	# トークンが語彙にない場合は[UNK]を使用
        token_ids.append(word2idx.get(t, word2idx["[UNK]"])) # word2idx["[UNK]"]：unknownのトークンを設定する引数

	  # もしtoken_idsがmax_lenより短い場合、[PAD]を追加してシーケンスを埋める
    if len(token_ids) < max_len:
        token_ids += [word2idx["[PAD]"]] * (max_len - len(token_ids)) # パッティングで足りない部分だけ埋める
    else:
	    	# 長すぎる場合はmax_lenに合わせてトランケート
        token_ids = token_ids[:max_len] # 無理やり区切る
    return token_ids

In [9]:
# トークン化された各文をencode関数を使ってエンコードし、encoded_datasetsに保存
encoded_datasets = []
for tokens in tokenized_texts:
    encoded_datasets.append(encode(tokens, word2idx, max_len=12))
print(encoded_datasets)

[[2, 7, 24, 15, 47, 20, 41, 3, 0, 0, 0, 0], [2, 7, 24, 15, 47, 3, 0, 0, 0, 0, 0, 0], [2, 7, 24, 15, 47, 3, 0, 0, 0, 0, 0, 0], [2, 8, 15, 47, 20, 3, 0, 0, 0, 0, 0, 0], [2, 10, 26, 12, 39, 21, 18, 22, 5, 3, 0, 0], [2, 11, 46, 42, 12, 31, 5, 32, 40, 29, 30, 28], [2, 9, 34, 44, 37, 25, 19, 14, 16, 38, 3, 0], [2, 6, 23, 45, 46, 27, 43, 12, 39, 17, 3, 0], [2, 7, 24, 15, 47, 20, 13, 41, 3, 0, 0, 0], [2, 7, 24, 15, 47, 20, 36, 35, 3, 0, 0, 0], [2, 7, 24, 15, 47, 20, 3, 0, 0, 0, 0, 0]]


In [10]:
# PyTorchのテンソルに変換
encoded_datasets = torch.tensor(encoded_datasets)
encoded_datasets.shape # [文の数, max_len]

torch.Size([11, 12])

In [12]:
# マスク関数を定義
# MLMのためにトークンをランダムに[MASK]で置き換える
# ただし、[PAD]、[CLS]、[SEP]、[MASK]などの特殊トークンはスキップする
def mask_tokens(batch, mask_prob=0.15):
    # バッチは2次元テンソルで、形状は(batch_size, sequence_length)
    # 元のデータをそのまま変更しないようにクローンする
    input_ids = batch.clone()

    # labelsは同じ形状のテンソルで、-100で埋める
    # CrossEntropyLossでignore_index=-100を指定すると、損失計算時に無視できる
    # マスクの部分だけEntropyLossで計算させたい
    labels = torch.full_like(batch, -100)

	  # 特殊トークンのIDを定義し、マスクしないようにする
    special_ids = {
        word2idx["[PAD]"],
        word2idx["[CLS]"],
        word2idx["[SEP]"],
        word2idx["[MASK]"]
    }

	  # バッチの各シーケンスとトークン位置をループして、マスクするかどうかを決定する　※今回は0～11まで回す
    for i in range(batch.size(0)):       # バッチ
        for j in range(batch.size(1)):   # トークン位置

			      # トークンIDを取得
            token_id = batch[i, j].item()

	      		# 特殊トークンの場合はスキップ
            if token_id in special_ids:
                continue

	      		# 15%の確率でそのトークンを[MASK]に置き換える
            if random.random() < mask_prob:
                input_ids[i, j] = word2idx["[MASK]"]
		          	# 置き換えた場所の正解ラベルを保持（それ以外は-100で損失計算時に無視される）
                labels[i, j] = token_id

	  # マスクされたトークンのIDと正解ラベルを返す
    return input_ids, labels

## BERTの実装

In [13]:
# BERT：特徴量の抽出に特化しているのでencoderしか使っていないモデル
# BERT用のMultiHeadSelfAttentionクラスを定義

# 1. MultiHeadSelfAttention: Transformerで使われる自己注意機構
# 2. TransformerEncoderBlock: 自己注意機構とフィードフォワードネットワークを組み合わせたブロック
# 3. PositionalEncoding: トークンの位置情報を加えるためのサイン波とコサイン波を使ったエンコーディング
# 4. MiniBert: 複数のエンコーダーブロックを積み重ねた小型モデル

# (A) MultiHeadSelfAttention
# このレイヤーは、文中の各トークンが他のトークンにどれだけ「注意」を払うべきかを計算する。
# 重要な関係に焦点を当てます。複数の「ヘッド」を使うことで、各ヘッドが異なる関係パターンを学習できる

from torch import nn

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads): # headの数が多いほど学習できる
        super().__init__()

	    	# 入力の埋め込みの次元数
        self.embed_dim = embed_dim

	    	# アテンションヘッドの数
        self.num_heads = num_heads

	    	# それぞれのヘッドの埋め込みの次元数は、入力の埋め込み次元をnum_headsで割った数
        self.head_dim = embed_dim // num_heads # headが増えても計算量は増えないように

	    	# クエリ、キー、バリューにそれぞれ重みがあるため、3つの線形変換を定義
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)

    		# 出力の重みがあるため、最終的な線形変換を定義
        self.out = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
	    	# Bはバッチサイズ、Sはシーケンスの長さ、Eは埋め込みの次元数
        B, S, E = x.size() # B: batch(64), S: squence(12), E: embedding(768)

		    # セルフアテンションのため、クエリ、キー、バリューは同じ入力 [B, S, E] -> [B, S, E]
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)

		    # マルチヘッドアテンション用にテンソルの形状を変える [B, S, E] -> [B, num_heads, S, head_dim]
        Q = Q.view(B, S, self.num_heads, self.head_dim).transpose(1, 2)
        K = K.view(B, S, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(B, S, self.num_heads, self.head_dim).transpose(1, 2)

        # Kの転地を取り、Qと掛け算することで、各トークンが他のトークンにどれだけ注意を払うべきかを計算
        # そして、head_dimの平方根でスケーリングすることで、数値の安定性を保つ
        # [B, num_heads, S, head_dim] * [B, num_heads, head_dim, S] -> [B, num_heads, S, S]
        # 論文で示された式(https://arxiv.org/abs/1706.03762)
        scores = torch.matmul(Q, K.transpose(-1, -2)) / math.sqrt(self.head_dim)

	    	# Softmaxを使って、スコアを確率に変換
        attn_weights = torch.softmax(scores, dim=-1)

	    	# 確率をバリューに掛け算することで、各トークンの重要度を考慮したコンテキストを得る
        context = torch.matmul(attn_weights, V)

        # Now, context is still in a multi-head form. We reorder it back to (B, S, E).
		    # コンテキストは[B, num_heads, S, head_dim]の形をしている
	    	# 最初に転地をし[B, S, num_heads, head_dim]にし、次に連結して[B, S, E]の形にする
        context = context.transpose(1, 2).contiguous().view(B, S, E)

	    	# 最終的な線形変換を通して、全てのヘッドからの情報を統合する
        out = self.out(context)
        return out

In [14]:
# TransformerEncoderBlock
# Transformerエンコーダーの1層。マルチヘッドアテンション、次にフィードフォワードサブレイヤーがあり、
# 各サブレイヤーの周りに残差接続と正規化がある。
class TransformerEncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout=0.1):
        super().__init__()
	    	# トークン間の関係を学習するためのマルチヘッドアテンション
        self.attn = MultiHeadSelfAttention(embed_dim, num_heads)

	    	# 各トークンの埋め込みを正規化することで、トレーニングを安定させるレイヤー正規化層
        self.norm1 = nn.LayerNorm(embed_dim)

	    	# 2つの線形変換を使ったフィードフォワードネットワーク
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim)
        )

		    # フィードフォワードネットワークの出力を正規化するためのレイヤー正規化層
        self.norm2 = nn.LayerNorm(embed_dim)

	      # 過学習を防ぐために、アクティベーションの一部をランダムにゼロにするドロップアウト
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # セルフアテンションを通して、トークン間の関係を学習（単語同士の関係性）
        # [B, S, E] -> [B, S, E]
        attn_out = self.attn(x)

        # ドロップアウトを適用し、残渣接続を追加して、レイヤー正規化を行う
        # [B, S, E] + [B, S, E] -> [B, S, E]
        x = self.norm1(x + self.dropout(attn_out))

        # フィードフォワードネットワークを通して、各トークンの埋め込みを変換
        # [B, S, E] -> [B, S, E]
        ff_out = self.ff(x)

		    # もう一度ドロップアウトを適用し、残渣接続を追加して、レイヤー正規化を行う
        x = self.norm2(x + self.dropout(ff_out))
        return x

In [15]:
# 位置エンコーディング
# トランスフォーマーは、シーケンス内の各トークンの位置を本質的に知らないため、
# 各トークンの位置に関する情報を注入する「位置エンコーディング」を追加。
# サイン波とコサイン波を使って、異なる周波数で位置をエンコードする。

import math

class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=1000):
        super().__init__()
		    # 位置エンコーディングを保存するためのテンソルを作成
        pe = torch.zeros(max_len, embed_dim)

        # 各トークンの位置[0, 1, 2, ..., max_len-1]を作成し
        # [max_len, 1]
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

	    	# 周波数を設定するために使用するdiv_termを作成
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * -(math.log(10000.0) / embed_dim))

	    	# 偶数はサイン波、奇数はコサイン波を使用して位置をエンコード
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # register_bufferは、peが学習パラメータではなく、このモジュールに属し、
        # モデルと一緒にGPUに移動されることを保証するために使用
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
	    	# xの形状は(B, S, E)で、最初のS位置に位置エンコーディングを追加
        seq_len = x.size(1)
        x = x + self.pe[:, :seq_len, :]
        return x


In [16]:
# MiniBERT
# - トークンIDを埋め込むためのEmbeddingレイヤー
# - 位置情報を加えるためのPositionalEncoding
# - 複数のTransformerEncoderBlockを積み重ねたもの
# - 各トークン位置に対して語彙のロジットを出力する最終的な線形レイヤー(mlm_head)

class MiniBert(nn.Module):
    def __init__(
        self,
        vocab_size,
        embed_dim=64,   # それぞれのトークンの埋め込みの次元数
        num_heads=2,    # マルチヘッドセルフアテンションで使用するヘッドの数
        ff_dim=128,     # それぞれのエンコーダーブロックのフィードフォワード層の次元数
        num_layers=2,   # TransformerEncoderBlockの数
        max_len=12      # 位置埋め込みの最大長
    ):
        super().__init__()
		    # 「意味」や「特徴」を捉えるために各トークンをベクトルに変換
        self.token_embed = nn.Embedding(vocab_size, embed_dim)

    		# トークンの位置情報を加えるための位置エンコーディングモジュール
        self.pos_encoding = PositionalEncoding(embed_dim, max_len)

	    	# TransformerEncoderBlockのリスト
        self.blocks = nn.ModuleList([
            TransformerEncoderBlock(embed_dim, num_heads, ff_dim)
            for _ in range(num_layers)
        ])
        # The final linear layer for Masked Language Modeling: we predict the correct token out of vocab_size.

        # MLM用の最終的な線形レイヤー
        # 出力は辞書（vocabulary）のサイズに等しいロジット
        self.mlm_head = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        # トークンIDを埋め込みベクトルに変換
        # [B, S] -> [B, S, E]
        x = self.token_embed(x)

        # 位置エンコーディングを追加
        # [B, S, E] -> [B, S, E]
        x = self.pos_encoding(x)

	    	# それぞれのエンコーダーブロックを順番に通す
        for block in self.blocks:
            x = block(x)

    		# 最終的な線形レイヤーを通して、各トークン位置に対して語彙のロジットを出力
        logits = self.mlm_head(x)
        return logits


In [17]:
# MiniBERTのトレーニング

from torch import optim
import random

# ハイパーパラメータの設定
# - vocab_size: 語彙のサイズ
# - embed_dim: 各トークンの埋め込みの次元数
# - num_heads: マルチヘッドセルフアテンションで使用するヘッドの数
# - ff_dim: フィードフォワードサブレイヤーの次元数
# - num_layers: スタックするエンコーダーブロックの数
# - max_len: 最大シーケンス長
# 最後にGPUに送る

model = MiniBert(
    vocab_size=vocab_size,
    embed_dim=64,
    num_heads=2,
    ff_dim=128,
    num_layers=2,
    max_len=12
).to(device)

# 最適化関数はAdamを使用
# Adamは、各パラメータの学習率を動的に調整するアルゴリズムで、
# 勾配の1次モーメントと2次モーメントの推定に基づいている。
# ニューラルネットワークのトレーニングにおいて、よく使われるデフォルトの選択肢。
# lr=1e-3は、一般的な初期学習率。

optimizer = optim.Adam(model.parameters(), lr=1e-3) # 第一引数：調整するパラメーター

# 損失関数はCrossEntropyLossを使用
# CrossEntropyLossは、分類問題に使われる損失関数で、
# モデルの出力（logits）と正解ラベル（labels）を比較して、モデルの性能を評価
# 具体的には、モデルが予測した各トークンの確率分布と、正解のトークンのインデックスを比較
# これは、モデルが予測した確率分布が、正解のトークンの分布とどれだけ一致しているかを測定
# ignore_index=-100は、-100でラベル付けされた位置は損失計算に影響しないことを意味
# これにより、マスクされたトークンの位置に対してのみ損失を計算するために使用
criterion = nn.CrossEntropyLoss(ignore_index=-100) # 二乗誤差

# エポック数
# どのくらいの回数データセットを使用して学習するか
epochs = 100

# バッチサイズ
# 1回の学習で処理する文の数
batch_size = 64

# データセットサイズは文の数
# ミニバッチを作成するために取得
dataset_size = encoded_datasets.size(0)

# trainモードに切り替えることで、ドロップアウトなどのレイヤーがトレーニング時の動作をするようにする。
model.train()

for epoch in range(1, epochs + 1):
    total_loss = 0

	  # データセットのインデックスをシャッフルする
    indices = list(range(dataset_size))
    random.shuffle(indices)

	  # バッチサイズごとにデータを処理する
    for i in range(0, dataset_size, batch_size):
		    # シャッフルしたリストからバッチインデックスを取得
        batch_idx = indices[i:i+batch_size]
		    # encoded_datasetsからバッチインデックスに基づいて行を取得し、GPUに移動
        batch = encoded_datasets[batch_idx].to(device)

		    # マスクトークンを適用して、[MASK]トークンを含むinput_idsと正解ラベルlabelsを作成
        input_ids, labels = mask_tokens(batch)
        input_ids, labels = input_ids.to(device), labels.to(device)

        # input_idsをモデルに通して、語彙サイズのロジットを得る
        # [B, S] -> [B, S, vocab_size]
        logits = model(input_ids)

        # CrossEntropyLossを使用して損失を計算
        # ロジットを[B*S, vocab_size]の形にフラット化し、ラベルを[B*S]の形にする
        # これにより、各トークン位置が別々の分類問題として扱われる
        loss = criterion(logits.view(-1, vocab_size), labels.view(-1))


	    	# 逆伝播を行う前に、既存の勾配をゼロにする。（PyTorchはデフォルトで勾配を蓄積するため）
        optimizer.zero_grad()

	    	# 逆伝播で勾配を計算する
        loss.backward()

	    	# 勾配を使ってパラメータを更新する
        optimizer.step()

		    # 損失を累積して、あとでエポック全体の平均を計算する
        total_loss += loss.item()

	  # エポック全体の平均損失を計算
    avg_loss = total_loss / (dataset_size / batch_size)
    print(f"Epoch [{epoch}/{epochs}] - Loss: {avg_loss:.4f}")

Epoch [1/100] - Loss: 24.4430
Epoch [2/100] - Loss: 23.0113
Epoch [3/100] - Loss: 21.6758
Epoch [4/100] - Loss: 20.3355
Epoch [5/100] - Loss: 21.2654
Epoch [6/100] - Loss: 21.5591
Epoch [7/100] - Loss: 21.2449
Epoch [8/100] - Loss: 17.3930
Epoch [9/100] - Loss: 21.8566
Epoch [10/100] - Loss: 21.5743
Epoch [11/100] - Loss: 21.5716
Epoch [12/100] - Loss: 19.3741
Epoch [13/100] - Loss: 19.9088
Epoch [14/100] - Loss: 18.2894
Epoch [15/100] - Loss: 21.1580
Epoch [16/100] - Loss: 21.2595
Epoch [17/100] - Loss: 23.4001
Epoch [18/100] - Loss: 17.8794
Epoch [19/100] - Loss: 19.8632
Epoch [20/100] - Loss: 19.3636
Epoch [21/100] - Loss: 19.0176
Epoch [22/100] - Loss: 18.9165
Epoch [23/100] - Loss: 20.1735
Epoch [24/100] - Loss: 21.3435
Epoch [25/100] - Loss: 18.6188
Epoch [26/100] - Loss: 19.0118
Epoch [27/100] - Loss: 20.5975
Epoch [28/100] - Loss: 18.8136
Epoch [29/100] - Loss: 21.0913
Epoch [30/100] - Loss: 21.6735
Epoch [31/100] - Loss: 17.6368
Epoch [32/100] - Loss: 20.5881
Epoch [33/100] - 

In [18]:
# 最後に評価

# 例えば、"hello how are [MASK] doing today"という文を使って、
# モデルが"[MASK]"を"you"と予測できるかを確認する

def encode_for_test(sentence, word2idx, max_len=12):
  	# 文の中の句読点を削除する
    for p in [".", ",", "?", "!", ":", ";"]:
        sentence = sentence.replace(p, "")

	# 文をスペースで分割してトークン化する
    tokens = sentence.strip().split()

	# [CLS]は文の開始を示し、[SEP]は文の終了を示す
    tokens = ["[CLS]"] + tokens + ["[SEP]"]
    token_ids = []

	# それぞれのトークンを整数IDに変換、もしくは語彙にない場合は[UNK]を使用
    for t in tokens:
        token_ids.append(word2idx.get(t, word2idx["[UNK]"]))

	# もしtoken_idsがmax_lenより短い場合、[PAD]を追加してシーケンスを埋める
    if len(token_ids) < max_len:
        token_ids += [word2idx["[PAD]"]] * (max_len - len(token_ids))
    else:
        token_ids = token_ids[:max_len]

	# 2次元のテンソルに変換して、バッチサイズ1としてモデルに渡す
    return torch.tensor(token_ids).unsqueeze(0)

# [MASK]を含む文を定義
test_sentence = "hello how are [MASK] doing today"

# モデルを評価モードに切り替える。これにより、ドロップアウトや他のトレーニング特有の動作がオフになる。
model.eval()

# torch.no_grad()は、勾配計算を無効にするためのコンテキストマネージャーで、
# モデルの評価時に使用される。これにより、メモリと計算を節約できる。
with torch.no_grad():

	  # test sentenceをトークンIDにエンコード
    test_ids = encode_for_test(test_sentence, word2idx).to(device)

	  # モデルに入力して、語彙サイズのロジットを得る
    logits = model(test_ids)

    # [MASK]の位置を見つける
    # test_ids[0]は1次元テンソルで、word2idx["[MASK]"]と比較してTrue/Falseを返す
    mask_idx = (test_ids[0] == word2idx["[MASK]"]).nonzero(as_tuple=True)

	  # もし[MASK]が1つ以上ある場合
    if len(mask_idx[0]) > 0:

	    	# 最初の[MASK]のインデックスを取得
        pos = mask_idx[0].item()

		    # logitsは[B, S, vocab_size]の形をしているので、最初のバッチの特定の位置のロジットを取得
        mask_logits = logits[0, pos, :]

		    # topkで、最も高い5つのロジットを取得
        top_values, top_indices = torch.topk(mask_logits, 5)

        print("Input:", test_sentence)
        print("Top predictions for [MASK]:")

	    	# トークンIDをトークン文字列にマッピングして、ロジットを表示
        for i in range(5):
            pred_id = top_indices[i].item()
            pred_word = idx2word[pred_id]
            print(f"  {pred_word} (logit: {top_values[i].item():.2f})")
    else:
        print("No [MASK] found.")

Input: hello how are [MASK] doing today
Top predictions for [MASK]:
  you (logit: 4.84)
  doing (logit: 3.18)
  are (logit: 1.84)
  a (logit: 0.91)
  again (logit: 0.73)
